In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ExplicitModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
# activation = F.relu
# architecture = [784, 128, 64, 10]
# lambda_ = 0.99999
# epsilon = 0.2
# supervised_lambda_weight = 1e-3
# neural_lr_start = 0.001 
# neural_lr_stop = 0.0005 
# neural_lr_rule = "constant"
# neural_lr_decay_multiplier = 0.005
# neural_dynamic_iterations = 50
activation = F.relu
architecture = [784, 500, 64, 10]
lambda_ = 0.99999
epsilon = 0.01
supervised_lambda_weight = 1e-3
neural_lr_start = 0.1 
neural_lr_stop = 0.0005 
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMaxNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
                         sgd_nesterov = False, optimizer_type = "sgd", 
                         optim_lr = 1, stepLR_step_size = 10*3000,)

In [ ]:
trn_acc_list = []
tst_acc_list = []
random_sign = False
n_epochs = 50

for epoch_ in range(n_epochs):
    Bcopy = torch.clone(model.B[0]["weight"])
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.to(device).view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        if random_sign:
            rnd_sgn = 2*np.random.randint(2) - 1
            supervised_lambda_weight = rnd_sgn * supervised_lambda_weight

        model.batch_step(  x, y_one_hot, supervised_lambda_weight,
                           neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                        )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
    print("B_1 update difference : {}".format(torch.norm(model.B[0]['weight'] - Bcopy)))

3000it [03:33, 14.07it/s]
1it [00:00,  8.23it/s]

Epoch : 1, Train Accuracy : 0.9461333333333334, Test Accuracy : 0.9451
B_1 update difference : 4.447171688079834


3000it [03:33, 14.04it/s]
1it [00:00,  8.15it/s]

Epoch : 2, Train Accuracy : 0.9618833333333333, Test Accuracy : 0.9594
B_1 update difference : 2.584958791732788


3000it [03:34, 14.01it/s]
1it [00:00,  7.99it/s]

...

3000it [04:47, 10.42it/s]
1it [00:00,  8.15it/s]

Epoch : 39, Train Accuracy : 0.9962833333333333, Test Accuracy : 0.9821
B_1 update difference : 3.7028799057006836


3000it [04:47, 10.44it/s]
1it [00:00,  7.71it/s]

Epoch : 40, Train Accuracy : 0.9962333333333333, Test Accuracy : 0.9823
B_1 update difference : 3.7647247314453125


3000it [03:46, 13.22it/s]


In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'CorInfoMax Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'CorInfoMax Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [8]:
plt.imshow(torch2numpy(model.B[0]['weight']))

In [9]:
torch2numpy(model.B[0]['weight'])

array([[ 5.716399  , -0.049982  , -0.03929942, ...,  0.22322418,
        -0.36075032,  0.06415747],
       [-0.049982  ,  5.679263  ,  0.41047487, ...,  0.36694136,
         0.09405217, -0.12705436],
       [-0.03929942,  0.41047487,  5.765897  , ..., -0.08526605,
        -0.20707147, -0.0348618 ],
       ...,
       [ 0.22322418,  0.36694136, -0.08526605, ...,  3.6762593 ,
        -0.27690554, -0.02166144],
       [-0.36075032,  0.09405217, -0.20707147, ..., -0.27690554,
         6.023473  , -0.27266186],
       [ 0.06415747, -0.12705436, -0.0348618 , ..., -0.02166144,
        -0.27266186,  5.8084693 ]], dtype=float32)

In [17]:
torch.svd(model.Wff[0]['weight'])[2].T

tensor([[-0.0003, -0.0213, -0.0094,  ..., -0.0170, -0.0083,  0.0116],
        [-0.0115, -0.0043,  0.0385,  ..., -0.0256, -0.0101,  0.0402],
        [ 0.0062, -0.0026, -0.0038,  ..., -0.0062,  0.0312, -0.0062],
        ...,
        [-0.0327,  0.0172,  0.0859,  ..., -0.0274, -0.0077,  0.1085],
        [-0.0180, -0.0393, -0.0425,  ..., -0.0113,  0.0153,  0.0442],
        [-0.0353, -0.0339,  0.0746,  ..., -0.0232,  0.0089,  0.0485]],
       device='cuda:0', grad_fn=<PermuteBackward0>)

In [ ]:
activation = F.relu
architecture = [784, 128, 64, 10]
lambda_ = 0.9999
epsilon = 0.01
supervised_lambda_weight = 1e-3
neural_lr_start = 0.001 
neural_lr_stop = 0.0005 
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMaxNudged(architecture, lambda_, epsilon, activation, use_stepLR = True, 
                         sgd_nesterov = False, optimizer_type = "sgd", 
                         optim_lr = 1e-3, stepLR_step_size = 10*3000,)

x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.to(device).view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

neurons = model.fast_forward(x, no_grad = True)
model.run_neural_dynamics(x, y_one_hot, neurons, supervised_lambda_weight, 
                          neural_lr_start, neural_lr_stop, lr_rule = neural_lr_rule, 
                          lr_decay_multiplier = neural_lr_decay_multiplier, 
                          neural_dynamic_iterations = neural_dynamic_iterations)

In [ ]:
mbs = x.size(1)
for jj in range(len(neurons)):
    neurons[jj] = neurons[jj].requires_grad_()
corinfo_cost = model.CorInfo_Cost(x, y, neurons)
init_grads = torch.tensor([1 for i in range(mbs)], dtype=torch.float, device=device, requires_grad=True) #Initializing gradients
grads = torch.autograd.grad(corinfo_cost, neurons, grad_outputs=init_grads, create_graph=False) # dPhi/ds

In [ ]:
one_over_epsilon = model.one_over_epsilon
gam_ = model.gam_

x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.to(device).view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

Wff = model.Wff
B = model.B

neurons = model.fast_forward(x, no_grad = True)

layers = [x] + neurons
for jj in range(len(Wff)):
    if jj == 0:
        error = - one_over_epsilon * (layers[jj + 1] - (Wff[jj]['weight'] @ layers[jj] + Wff[jj]['bias'])) 
    else:
        error = - one_over_epsilon * (layers[jj + 1] - (Wff[jj]['weight'] @ model.activation(layers[jj]) + Wff[jj]['bias']))

    lateral_term = gam_ * 0.5 * (layers[jj + 1].T @ B[jj]['weight'] @ layers[jj + 1])
    corinfo_cost = torch.sum(error * error, 0)
    


In [ ]:
error.shape
torch.sum(error * error, 0).shape

In [ ]:
outer_prod_broadcasting((B[jj]['weight'] @ layers[jj + 1]), layers[jj + 1].T).shape

In [ ]:
layers[jj + 1][:,2].T @ B[jj]['weight'] @ layers[jj + 1][:,2]

In [ ]:
torch.sum((B[jj]['weight'] @ layers[jj + 1]) * layers[jj + 1], 0)

In [ ]:
(B[jj]['weight'] @ layers[jj + 1]).shape, layers[jj + 1].shape

In [ ]:
trn_acc_list = []
tst_acc_list = []
random_sign = False
n_epochs = 50

for epoch_ in range(n_epochs):
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.to(device).view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        if random_sign:
            rnd_sgn = 2*np.random.randint(2) - 1
            supervised_lambda_weight = rnd_sgn * supervised_lambda_weight

        model.batch_step(  x, y_one_hot, supervised_lambda_weight,
                           neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                        )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))